In [1]:
import requests
import json
import pandas as pd
from IPython.display import display, HTML
from bs4 import BeautifulSoup

In [2]:
# Cấu hình Moodle API
moodle_url = "http://localhost:8100/webservice/rest/server.php"
api_token = "ed3520980c2a735542b59da43e98188f"

def call_moodle_api(wsfunction, params=None):
    """
    Gọi Moodle Web Service API với hàm và tham số được chỉ định.

    Args:
        wsfunction (str): Tên hàm Web Service.
        params (dict, optional): Tham số bổ sung cho API.

    Returns:
        dict or list: Dữ liệu JSON trả về từ API hoặc None nếu có lỗi.
    """
    default_params = {
        'wstoken': api_token,
        'wsfunction': wsfunction,
        'moodlewsrestformat': 'json'
    }
    if params:
        default_params.update(params)
    
    try:
        response = requests.get(moodle_url, params=default_params)
        if response.status_code != 200:
            print(f"❌ Lỗi HTTP: {response.status_code}")
            print("Nội dung phản hồi:", response.text)
            return None
        
        data = response.json()
        if isinstance(data, dict) and 'exception' in data:
            print(f"❌ Lỗi API: {data['errorcode']}")
            print(f"Thông điệp: {data['message']}")
            return None
        
        return data
    
    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi khi gửi yêu cầu: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ Lỗi phân tích JSON: {e}")
        print("Nội dung phản hồi:", response.text)
        return None

def clean_html(html_text):
    """
    Làm sạch HTML để lấy văn bản thuần túy.

    Args:
        html_text (str): Chuỗi HTML.

    Returns:
        str: Văn bản đã làm sạch.
    """
    return BeautifulSoup(html_text, 'html.parser').get_text() if html_text else ""

def get_all_courses():
    """
    Lấy danh sách tất cả các khóa học.

    Returns:
        list: Danh sách các khóa học hoặc None nếu có lỗi.
    """
    data = call_moodle_api('core_course_get_courses')
    return data if data else []

def get_course_contents(courseid):
    """
    Lấy chi tiết nội dung của một khóa học, bao gồm section và module.

    Args:
        courseid (int): ID của khóa học.

    Returns:
        list: Danh sách các section hoặc None nếu có lỗi.
    """
    return call_moodle_api('core_course_get_contents', {'courseid': courseid})

def get_lesson_details(lessonid):
    """
    Lấy chi tiết một bài học (lesson).

    Args:
        lessonid (int): ID của bài học.

    Returns:
        dict: Thông tin bài học hoặc None nếu có lỗi.
    """
    return call_moodle_api('mod_lesson_get_lesson', {'lessonid': lessonid})

def get_lesson_pages(lessonid):
    """
    Lấy danh sách các trang trong một bài học.

    Args:
        lessonid (int): ID của bài học.

    Returns:
        list: Danh sách các trang hoặc None nếu có lỗi.
    """
    return call_moodle_api('mod_lesson_get_pages', {'lessonid': lessonid})

def build_course_structure(courseid):
    """
    Xây dựng cấu trúc lồng nhau của một khóa học.

    Args:
        courseid (int): ID của khóa học.

    Returns:
        dict: Cấu trúc lồng nhau của khóa học.
    """
    course_data = get_course_contents(courseid)
    if not course_data:
        return None
    
    course_structure = {
        'courseid': courseid,
        'sections': []
    }
    
    for section in course_data:
        section_info = {
            'id': section['id'],
            'name': section.get('name', 'Không có tên'),
            'summary': clean_html(section.get('summary', '')),
            'subsections': [],
            'modules': []
        }
        
        for module in section['modules']:
            module_info = {
                'id': module['id'],
                'name': module['name'],
                'type': module['modname'],
                'instance': module.get('instance', 0),
                'url': module.get('url', '')
            }
            
            if module['modname'] == 'subsection':
                # Subsection là một module đặc biệt, lấy nội dung con
                subsection_data = get_course_contents(module['instance'])
                if subsection_data:
                    subsection_info = {
                        'id': module['instance'],
                        'name': module['name'],
                        'summary': clean_html(section.get('summary', '')),
                        'modules': []
                    }
                    for sub_section in subsection_data:
                        for sub_module in sub_section['modules']:
                            sub_module_info = {
                                'id': sub_module['id'],
                                'name': sub_module['name'],
                                'type': sub_module['modname'],
                                'instance': sub_module.get('instance', 0),
                                'url': sub_module.get('url', ''),
                                'content': []
                            }
                            if sub_module['modname'] == 'lesson':
                                lesson_data = get_lesson_details(sub_module['instance'])
                                if lesson_data:
                                    sub_module_info['lesson_details'] = {
                                        'name': lesson_data.get('name', ''),
                                        'intro': clean_html(lesson_data.get('intro', ''))
                                    }
                                    pages = get_lesson_pages(sub_module['instance'])
                                    if pages and isinstance(pages, list):
                                        sub_module_info['content'] = [
                                            {
                                                'page_id': page['page']['id'],
                                                'title': page['page']['title'],
                                                'type': page['page']['type'],
                                                'contents': clean_html(page['page'].get('contents', ''))
                                            }
                                            for page in pages
                                        ]
                            subsection_info['modules'].append(sub_module_info)
                    section_info['subsections'].append(subsection_info)
            else:
                if module['modname'] == 'lesson':
                    lesson_data = get_lesson_details(module['instance'])
                    if lesson_data:
                        module_info['lesson_details'] = {
                            'name': lesson_data.get('name', ''),
                            'intro': clean_html(lesson_data.get('intro', ''))
                        }
                        pages = get_lesson_pages(module['instance'])
                        if pages and isinstance(pages, list):
                            module_info['content'] = [
                                {
                                    'page_id': page['page']['id'],
                                    'title': page['page']['title'],
                                    'type': page['page']['type'],
                                    'contents': clean_html(page['page'].get('contents', ''))
                                }
                                for page in pages
                            ]
                section_info['modules'].append(module_info)
        
        course_structure['sections'].append(section_info)
    
    return course_structure

def display_course_structure(course_structure):
    """
    Hiển thị cấu trúc khóa học dưới dạng bảng và JSON.

    Args:
        course_structure (dict): Cấu trúc lồng nhau của khóa học.
    """
    print(f"\n=== Cấu trúc khóa học (courseid={course_structure['courseid']}) ===")
    print(json.dumps(course_structure, indent=2, ensure_ascii=False))
    
    for section in course_structure['sections']:
        print(f"\n📚 Section: {section['name']} (ID: {section['id']})")
        print(f"Tóm tắt: {section['summary']}")
        
        if section['subsections']:
            print("\n🔖 Subsections:")
            for subsection in section['subsections']:
                print(f"  - Subsection: {subsection['name']} (ID: {subsection['id']})")
                print(f"    Tóm tắt: {subsection['summary']}")
                if subsection['modules']:
                    module_data = [
                        {
                            'Module ID': m['id'],
                            'Tên': m['name'],
                            'Loại': m['type'],
                            'Nội dung': m.get('lesson_details', {}).get('intro', '') or m.get('content', [''])[0].get('contents', '')
                        }
                        for m in subsection['modules']
                    ]
                    df = pd.DataFrame(module_data)
                    display(HTML(df.to_html(index=False)))
        
        if section['modules']:
            print("\n📖 Modules:")
            module_data = [
                {
                    'Module ID': m['id'],
                    'Tên': m['name'],
                    'Loại': m['type'],
                    'Nội dung': m.get('lesson_details', {}).get('intro', '') or m.get('content', [''])[0].get('contents', '')
                }
                for m in section['modules']
            ]
            df = pd.DataFrame(module_data)
            display(HTML(df.to_html(index=False)))

# Gọi hàm để lấy và hiển thị cấu trúc khóa học
if __name__ == "__main__":
    courseid = 3  # Khóa học "NGÔN NGỮ LẬP TRÌNH JAVA"
    # In ra kết quả dễ đọc
    print(f"=== Lấy cấu trúc khóa học (courseid={courseid}) ===")
    data = get_course_contents(courseid)
    print(json.dumps(data, indent=2, ensure_ascii=False))
    # course_structure = build_course_structure(courseid)
    # if course_structure:
    #     display_course_structure(course_structure)

=== Lấy cấu trúc khóa học (courseid=3) ===
[
  {
    "id": 9,
    "name": "General",
    "visible": 1,
    "summary": "",
    "summaryformat": 1,
    "section": 0,
    "hiddenbynumsections": 0,
    "uservisible": true,
    "component": null,
    "itemid": null,
    "modules": [
      {
        "id": 9,
        "url": "http://localhost:8100/mod/forum/view.php?id=9",
        "name": "Announcements",
        "instance": 2,
        "contextid": 31,
        "visible": 1,
        "uservisible": true,
        "visibleoncoursepage": 1,
        "modicon": "http://localhost:8100/theme/image.php/boost/forum/1752649332/monologo?filtericon=1",
        "modname": "forum",
        "purpose": "collaboration",
        "branded": false,
        "modplural": "Forums",
        "availability": null,
        "indent": 0,
        "onclick": "",
        "afterlink": null,
        "activitybadge": [],
        "customdata": "{\"trackingtype\":\"1\"}",
        "noviewlink": false,
        "completion": 0,
      